In [ ]:
import numpy as np
from correlation_torch import NormXCorr2
import matplotlib.pyplot as plt
import cv2
import time
import torch
from compute_offset import ApplyShifts
from LoadData import load_data as ld
from scipy.io import loadmat
import scipy.io as io
import torch.nn as nn
from torch.nn import functional as F
from NCCRegistration import NCCMotionCorrection as NMC

In [ ]:
path="msCam1.avi"#replace by your file path
video,_,_=ld(path).read_file()
video=torch.tensor(video, dtype=torch.float32)
use_gpu=True
CROP=False #After testing， cropping function is abandoned in the latest code. So please set CROP False. 
#Crop frame to reduce computation pixels. If the user has high-configured GPU, CROP can be False

#create the kernel of high-pass filter
kernel=ld(path).generate_kernel((10,10))
kernel=torch.tensor(kernel, dtype=torch.float32)

if CROP==True:
    crop_size=150 #This size should be smaller than the height and width of the video
    crop_frame=torch.zeros([crop_size,crop_size], dtype=torch.float)
    sum1, a_rot_complex, b_complex, Zeros, theta, template_buffer=ld(path).SetParameters(crop_frame, kernel, CROP, use_gpu)
else:
    crop_size=0
    sum1, a_rot_complex, b_complex, Zeros, theta, template_buffer=ld(path).SetParameters(video[:,:,0], kernel, CROP, use_gpu)

new_video=torch.empty([video.shape[0], video.shape[1], video.shape[2]])
y_shift=torch.empty([1,video.shape[2]], dtype=torch.float)
x_shift=torch.empty([1,video.shape[2]], dtype=torch.float)

if use_gpu==True:
    kernel=kernel.cuda()
    y_shift=y_shift.cuda()
    x_xshift=x_shift.cuda()

In [ ]:
#Generate template for registration by using the initial 100 frames
init_batch=100
init_video=video[:,:,0:100]
motion_correction=NMC(sum1, a_rot_complex, b_complex, Zeros, theta, template_buffer, crop_size)
preprocess_temp=motion_correction.generate_template(init_batch, init_video, kernel, use_gpu, CROP)
del init_video

In [ ]:
#Begin motion correction
start=time.clock()
new_video, x_shift, y_shift=motion_correction.Online_NCC_motion_correction(video, new_video, x_shift, y_shift, preprocess_temp,  kernel, use_gpu, CROP)
end=time.clock()-start
print('Time consumption of registration:', end, 's')

In [ ]:
if use_gpu==True:
    x_shift=x_shift.cpu()
    y_shift=y_shift.cpu()

x_shift=x_shift.numpy()
y_shift=y_shift.numpy()


In [ ]:
x=range(x_shift.shape[1])
plt.plot(x,y_shift[0,:], label='shifts on y-axis')
plt.plot(x,x_shift[0,:], label='shifts on x-axis')
plt.legend()
plt.show()


In [ ]:
video=video.numpy()
new_video=new_video.numpy()
new_video=np.array(new_video, dtype='uint8')
save_movie=True
if save_movie:
    save_name='mcvideo'
    io.savemat(save_name, {'new_video': new_video})